In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from itertools import combinations
from tqdm import tqdm
from hapi import *
db_begin('data')
%matplotlib inline

HAPI version: 1.2.2.0
To get the most up-to-date version please check http://hitran.org/hapi
ATTENTION: Python versions of partition sums from TIPS-2021 are now available in HAPI code

           MIT license: Copyright 2021 HITRAN team, see more at http://hitran.org. 

           If you use HAPI in your research or software development,
           please cite it using the following reference:
           R.V. Kochanov, I.E. Gordon, L.S. Rothman, P. Wcislo, C. Hill, J.S. Wilzewski,
           HITRAN Application Programming Interface (HAPI): A comprehensive approach
           to working with spectroscopic data, J. Quant. Spectrosc. Radiat. Transfer 177, 15-30 (2016)
           DOI: 10.1016/j.jqsrt.2016.03.005

           ATTENTION: This is the core version of the HITRAN Application Programming Interface.
                      For more efficient implementation of the absorption coefficient routine, 
                      as well as for new profiles, parameters and other functional,
      

In [2]:
comps = ['H2O', 'O3', 'O2', 'CH4', 'SO2', 'NH3', 'NO', 'H2S']
isos = [(1,1),(3,1),(7,1),(6,1),(9,1),(11,1),(8,1),(31,1)]

combisos2 = itertools.combinations(isos, 2)
combcomps2 = itertools.combinations(comps, 2)
comb_isos2 = []
for comb in combisos2:
    comb_isos2.append(list(comb))
comb_comps2 = []
for comb in combcomps2:
    comb_comps2.append(list(comb))

'''combisos3 = itertools.combinations(isos, 3)
combcomps3 = itertools.combinations(comps, 3)
comb_isos3 = []
for comb in combisos3:
    comb_isos3.append(list(comb))
comb_comps3 = []
for comb in combcomps3:
    comb_comps3.append(list(comb))'''

'''combisos4 = itertools.combinations(isos, 4)
combcomps4 = itertools.combinations(comps, 4)
comb_isos4 = []
for comb in combisos4:
    comb_isos4.append(list(comb))
comb_comps4 = []
for comb in combcomps4:
    comb_comps4.append(list(comb)) 

combisos5 = itertools.combinations(isos, 5)
combcomps5 = itertools.combinations(comps, 5)
comb_isos5 = []
for comb in combisos5:
    comb_isos5.append(list(comb))
comb_comps5 = []
for comb in combcomps5:
    comb_comps5.append(list(comb))
'''

'combisos4 = itertools.combinations(isos, 4)\ncombcomps4 = itertools.combinations(comps, 4)\ncomb_isos4 = []\nfor comb in combisos4:\n    comb_isos4.append(list(comb))\ncomb_comps4 = []\nfor comb in combcomps4:\n    comb_comps4.append(list(comb)) \n\ncombisos5 = itertools.combinations(isos, 5)\ncombcomps5 = itertools.combinations(comps, 5)\ncomb_isos5 = []\nfor comb in combisos5:\n    comb_isos5.append(list(comb))\ncomb_comps5 = []\nfor comb in combcomps5:\n    comb_comps5.append(list(comb))\n'

In [3]:
def mix_spectra(compniso, comp, press):
    nu,coef = absorptionCoefficient_Voigt((compniso),comp,OmegaRange=(121, 125),OmegaStep=0.004, HITRAN_units=False, Environment={'p':press/10,'T':296}, Diluent={"self": 1}, Verbose=False) # compniso - кортеж состоящий из номеров изотопов молекул смеси, comp - молекулы смеси
    return coef

In [5]:
#If the CO2 content is 100%, set the 'SELF': 1, and another parameter setting Air = 0
#If the CO2 content is 1%, the settings of 'H2O' = 0.01, and the other set to 'Air' = 0.99

#If it is a pure air environment, set the 'Air' = 1

In [5]:
mixes = []
params2c1 = []
params2c2 = []
for i in range(len(comb_isos2)):
    for j in tqdm(range(1,11), desc=f"pressures for {comb_comps2[i]}"):
        mixes.append(mix_spectra(comb_isos2[i],comb_comps2[i],j))
        params2c1.append(comb_comps2[i])  # записываем параметры сгенерированной смеси, для обработки результатов. здесь - компоненты смеси
        params2c2.append(j/10) # давление смеси

pressures for ['NO', 'H2S']: 100%|██████████| 10/10 [00:09<00:00,  1.09it/s]


In [10]:
mixes

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
mixes=np.array(mixes)
mixes.shape

(280, 1001)

In [9]:
np.savetxt("mixes.txt", mixes, delimiter=" ")

In [7]:
temp1 = params2c1

In [8]:
y_ = [0,0,0,0,0,0,0,0]
y_true = [] # вектор правильных ответов, 1-компонент присутствует, 0 - отсутствует. для этого итерируемся по всему списку комбинаций
for i in range(280):
    z=temp1[i]
    for k in z:
        for i in range(len(y_)):
            if (comps[i])==k:
                y_[i] = 1
                y_true.append(y_)
    y_ = [0,0,0,0,0,0,0,0]
len(y_true)

560

In [9]:
y_true2 = []
for i in range(1,len(y_true),2):
    y_true2.append(y_true[i])
y_true2 = np.array(y_true2)
y_true2.shape

(280, 8)

In [10]:
np.savetxt("y_true2.txt", y_true2)

In [11]:
params2c1 = pd.DataFrame(params2c1)
params2c2 = pd.DataFrame(params2c2)

In [12]:
params2c1.to_csv('components.csv', index=False)
params2c2.to_csv('pressure.csv', index=False)